In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
print('Welcome to Elite Tickets.')

import mysql.connector as sqltr
#Please change the MySQL password
con=sqltr.connect(host='localhost',user='root',password='soumya126',database='project')
if con.is_connected==False:
    print('Error in connection')
cursor=con.cursor()

while True:
    print('Date: 20 September, 2020')
    print('Day: Sunday')
    print('''Movies available:''')
    mdf=pd.DataFrame({'Movie ID':['E1493','A3032','E4523','E5099','A6041'],'Movie Name':['Movie1','Movie2','Movie3','Movie4','Movie5'],'Theatre No':['T1','T3','T4','T5','T6'],'Age Rating':['R','PG','PG','PG-13','PG'],'Language':['English','Arabic','English','English','Arabic'],'Morning':['8:00','8:15','8:45','9:15','9:45'],'Afternoon':['12:15','12:45','13:00','14:15','14:45'],'Evening':['16:30','16:45','17:00','17:30','17:45'],'Night':['20:00','20:30','20:45','21:15','21:45']},index=['i)','ii)','iii)','iv)','v)'])
    display(mdf)

    print('''1) Book tickets
2) View Booking Details
3) Edit Booking Details
4) Cancel Booking
5) Popularity''')
    print()
    choice=input('Enter your choice(1,2,3,4,5): ')
    print()

    if choice=='1':
        print('Booking tickets')
        while True:
            info=pd.read_sql_query("Select * from ticket;",con)
            movieg=info.groupby('MovieID')
            QID=int(input('Enter your Qatar ID number: '))
            MID=input('Enter your movie ID: ')
            while True:
                try:
                    moviedf=movieg.get_group(MID)
                except KeyError:
                    if MID=='E1493' or MID=='A3032' or MID=='E4523' or MID=='E5099' or MID=='A6041':
                        BookedTickets=0
                        print('We have 200 seats remaining')
                        break
                    else:
                        print('Invalid movie ID')
                        MID=input('Enter a valid movie ID: ')
                        continue
                else:
                    BookedTickets=moviedf['NoOfTickets'].sum()
                    if BookedTickets==200:
                        print('No tickets available for',MID)
                        print('We have 0 seats remaining')
                        DiffBookQ=input('Do you want to make a booking for another movie?(y/n): ')
                        if DiffBookQ=='y' or DiffBookQ=='yes' or DiffBookQ=='Yes' or DiffBookQ=='Y' or DiffBookQ=='YES':
                            MID=input('Enter MovieID: ')
                            continue
                        else:
                            break
                    else:
                        print('We have',200-BookedTickets,'seats remaining')
                        break
            if BookedTickets==200:
                if DiffBookQ=='y' or DiffBookQ=='yes' or DiffBookQ=='Yes' or DiffBookQ=='Y' or DiffBookQ=='YES':
                    pass
                else:
                    break
            Name=input('Enter your name: ')
            PhoneNo=int(input('Enter your Phone number: '))
            NoOfTickets=int(input('Enter the number of tickets: '))
            while True:
                if BookedTickets+NoOfTickets>200:
                    print('Exceeded total number of theatre seats')
                    print('We only have',200-BookedTickets,'seats remaining')
                    NoOfTickets=int(input('Enter a valid number of tickets: '))
                elif NoOfTickets<1:
                    print('Invalid number of tickets')
                    NoOfTickets=int(input('Enter a valid number of tickets: '))
                else:
                    break
            Shift=input('Enter the shift(Morning,Afternoon,Evening,Night): ')
            Snacks=input('Enter preferred snacks(Popcorn,Nachos,Fries,Chips): ')
            cursor.execute("Insert into ticket values({},'{}','{}',{},{},'{}','{}')".format(QID,MID,Name,PhoneNo,NoOfTickets,Shift,Snacks))
            con.commit()
            if NoOfTickets>1:
                print('Tickets booked')
            else:
                print('Ticket booked')
            print()
            BookAQ=input('Do you want to make another booking?(y/n): ')
            if BookAQ=='y' or BookAQ=='yes' or BookAQ=='Yes' or BookAQ=='Y' or BookAQ=='YES':
                print()
                continue
            else:
                break

    elif choice=='2':
        print('Viewing Booking Details')
        info=pd.read_sql_query("Select * from ticket;",con)
        group=info.groupby(['QatarID','MovieID'])
        while True:
            QID=int(input('Enter your Qatar ID number: '))
            MID=input('Enter your movie ID: ')
            Specific=group.get_group((QID,MID))
            Specific.set_index('QatarID',inplace=True)
            print('Name          : ',Specific.Name[QID])
            print('Phone Number  : ',Specific.PhoneNo[QID])
            print('No. of tickets: ',Specific.NoOfTickets[QID])
            print('Shift         : ',Specific.Shift[QID])
            print('Snacks        : ',Specific.Snacks[QID])
            print()
            ViewAQ=input('Do you want to view details of a different booking?(y/n): ')
            if ViewAQ=='y' or ViewAQ=='yes' or ViewAQ=='Yes' or ViewAQ=='Y' or ViewAQ=='YES':
                print()
                continue
            else:
                break

    elif choice=='3':
        print('Editing Booking Details')
        QID=int(input('Enter your Qatar ID number: '))
        MID=input('Enter your movie ID: ')
        info=pd.read_sql_query("Select * from ticket;",con)        
        movieg=info.groupby('MovieID')        
        group=info.groupby(['QatarID','MovieID'])
        Specific=group.get_group((QID,MID))
        Specific.set_index('QatarID',inplace=True)
        print('1)Name          : ',Specific.Name[QID])
        print('2)Phone Number  : ',Specific.PhoneNo[QID])
        print('3)No. of tickets: ',Specific.NoOfTickets[QID])
        print('4)Shift         : ',Specific.Shift[QID])
        print('5)Snacks        : ',Specific.Snacks[QID])
        while True:
            Detail=input('Which detail do you want to edit?(1,2,3,4,5): ')
            if Detail=='1':
                col='Name'
                DN='Name'
            elif Detail=='2':
                col='PhoneNo'
                DN='Phone number'
            elif Detail=='3':
                col='NoOfTickets'
                DN='Number of tickets'
            elif Detail=='4':
                col='Shift'
                DN='Shift(Morning,Afternoon,Evening,Night)'
            elif Detail=='5':
                col='Snacks'
                DN='Snacks(Popcorn,Nachos,Fries,Chips)'
            else:
                print('Invalid Option')
                continue
            if Detail=='1' or Detail=='4' or Detail=='5':
                Change=input('Enter new value for '+DN+': ')
                cursor.execute("Update ticket set "+col+"='{}' where QatarID={} and MovieID='{}'".format(Change,QID,MID))
            elif Detail=='2':
                Change=int(input('Enter new value for '+DN+': '))
                cursor.execute("Update ticket set "+col+"={} where QatarID={} and MovieID='{}'".format(Change,QID,MID))
            elif Detail=='3':
                moviedf=movieg.get_group(MID)
                BookedTickets=moviedf['NoOfTickets'].sum()-Specific.NoOfTickets[QID]
                print('We have',200-BookedTickets,'seats remaining')
                Change=int(input('Enter new value for '+DN+': '))
                while True:
                    if BookedTickets+Change>200:
                        print('Exceeded total number of theatre seats')
                        print('We only have',200-BookedTickets,'seats remaining')
                        Change=int(input('Enter a valid number of tickets: '))
                    elif Change<1:
                        print('Invalid number of tickets')
                        Change=int(input('Enter a valid number of tickets: '))
                    else:
                        break
                cursor.execute("Update ticket set "+col+"={} where QatarID={} and MovieID='{}'".format(Change,QID,MID))
            else:
                pass
            con.commit()
            print('Updated details:')
            info=pd.read_sql_query("Select * from ticket;",con)
            group=info.groupby(['QatarID','MovieID'])
            Specific=group.get_group((QID,MID))
            Specific.set_index('QatarID',inplace=True)
            print('1)Name         : ',Specific.Name[QID])
            print('2)Phone Number : ',Specific.PhoneNo[QID])
            print('3)Total tickets: ',Specific.NoOfTickets[QID])
            print('4)Shift        : ',Specific.Shift[QID])
            print('5)Snacks       : ',Specific.Snacks[QID])
            print()
            EditAQ=input('Do you want to edit another detail?(y/n): ')
            if EditAQ=='y' or EditAQ=='yes' or EditAQ=='Yes' or EditAQ=='Y' or EditAQ=='YES':
                print()
                continue
            else:
                break

    elif choice=='4':
        print('Cancelling Booking')
        while True:
            QID=int(input('Enter your Qatar ID number: '))
            MID=input('Enter your movie ID: ')
            info=pd.read_sql_query("Select * from ticket;",con)
            group=info.groupby(['QatarID','MovieID'])
            Specific=group.get_group((QID,MID))
            Specific.set_index('QatarID',inplace=True)
            print('Name         : ',Specific.Name[QID])
            print('Phone Number : ',Specific.PhoneNo[QID])
            print('Total tickets: ',Specific.NoOfTickets[QID])
            print('Shift        : ',Specific.Shift[QID])
            print('Snacks       : ',Specific.Snacks[QID])
            CancelQ=input('Are you sure you want to cancel the above booking?(y/n): ')
            if CancelQ=='y'or CancelQ=='Y' or CancelQ=='Yes' or CancelQ=='yes' or CancelQ=='YES':
                cursor.execute("Delete from ticket where QatarID={} and MovieID='{}'".format(QID,MID))
                con.commit()
                print('Booking Cancelled')
            else:
                pass
            print()
            CAQ=input('Do you want to cancel another booking?(y/n): ')
            if CAQ=='y'or CAQ=='Y' or CAQ=='Yes' or CAQ=='yes' or CAQ=='YES':
                print()
                continue
            else:
                break

    elif choice=='5':
        print('Popularity')
        info=pd.read_sql_query("Select * from ticket;",con)
        while True:
            print('''1)Popularity of Snacks
2)Popularity of Movies
3)Rush Hours''')
            Pop=input('Which option do you want to see?(1,2,3): ')
            if Pop=='1':
                g=info.groupby('Snacks')
                Size=g.size()
                spie=plt.pie(g['NoOfTickets'].sum(),labels=Size.index,autopct='%0.2f%%')
                plt.title('Popularity of Snacks')
                plt.show()
            elif Pop=='2':
                g=info.groupby('MovieID')
                Size=g.size()
                mpie=plt.pie(g['NoOfTickets'].sum(),labels=Size.index,autopct='%0.2f%%')
                plt.title('Popularity of Movies')
                plt.show()
            elif Pop=='3':
                g=info.groupby('Shift')
                Size=g.size()
                xaxis=Size.index
                yaxis=g['NoOfTickets'].sum()
                rbar=plt.bar(xaxis,yaxis)
                plt.xlabel('Shift')
                plt.ylabel('No of People')
                plt.title('Rush Hours')
                plt.show()
            else:
                print('Invalid option')
            GAgainQ=input('Do you want to see another popularity option?(y/n): ')
            if GAgainQ=='y'or GAgainQ=='Y' or GAgainQ=='Yes' or GAgainQ=='yes' or GAgainQ=='YES':
                continue
            else:
                break

    else:
        print('Invalid Option')
    print()
    OverallAQ=input('Do you want to select another option?(y/n): ')
    if OverallAQ=='y'or OverallAQ=='Y' or OverallAQ=='Yes' or OverallAQ=='yes' or OverallAQ=='YES':
        print()
        continue
    else:
        break
   
con.close()

Welcome to Elite Tickets.


ModuleNotFoundError: ignored